In [ ]:
import os
import win32com.client

def update_cell_preserving_everything(folder_path):
    excel = win32com.client.Dispatch("Excel.Application")
    excel.Visible = False
    excel.DisplayAlerts = False

    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            wb = excel.Workbooks.Open(file_path)
            ws = wb.Sheets(1)  # First sheet
            ws.Range("A10").Value = "Setting for Sep'25"
            wb.Save()
            wb.Close()

    excel.Quit()

In [ ]:
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.workbook.defined_name import DefinedName

def sanitize_name(name: str, prefix: str) -> str:
    return f"{prefix}_{''.join(c if c.isalnum() else '_' for c in name.strip())}"

def update_cascading_dropdowns(file_path: str, selected_l2: str, wb, ws_options, ws_l2, ws_l3, ws_l4):
    col_l2, col_l3, col_l4 = "E", "F", "G"
    l2_to_l3, l3_to_l4 = {}, {}

    for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
        l2_val = row[ord(col_l2) - ord('A')].value
        l3_val = row[ord(col_l3) - ord('A')].value
        l4_val = row[ord(col_l4) - ord('A')].value

        if l2_val and l3_val:
            l2_to_l3.setdefault(str(l2_val).strip(), set()).add(str(l3_val).strip())
        if l3_val and l4_val:
            l3_to_l4.setdefault(str(l3_val).strip(), set()).add(str(l4_val).strip())

    selected_l2 = selected_l2.strip()
    matched_l3 = sorted(l2_to_l3.get(selected_l2, []))
    matched_l4 = sorted(set(l4 for l3 in matched_l3 for l4 in l3_to_l4.get(l3, [])))

    for col in ["A", "B", "C"]:
        for cell in ws_options[col][2:]:
            cell.value = None

    ws_options["A3"] = selected_l2
    for i, val in enumerate(matched_l3, start=3):
        ws_options[f"B{i}"] = val

    if matched_l3:
        name_l2 = sanitize_name(selected_l2, "L2")
        range_l3 = f"Options!$B$3:$B${2 + len(matched_l3)}"
        wb.defined_names.add(DefinedName(name=name_l2, attr_text=range_l3))

    start_row = 3
    for l3_val in matched_l3:
        l4_list = sorted(l3_to_l4.get(l3_val, []))
        for i, val in enumerate(l4_list, start=start_row):
            ws_options[f"C{i}"] = val
        name_l3 = sanitize_name(l3_val, "L3")
        range_l4 = f"Options!$C${start_row}:$C${start_row + len(l4_list) - 1}"
        wb.defined_names.add(DefinedName(name=name_l3, attr_text=range_l4))
        start_row += len(l4_list)

    if matched_l3:
        ws_l3["B1"] = matched_l3[0]
    if matched_l4:
        ws_l4["B1"] = matched_l4[0]

    ws_l2.unmerge_cells("B1:C1")
    ws_l2.merge_cells("B1:C1")
    ws_l2["B1"] = selected_l2

    ws_l3.unmerge_cells("B1:C1")
    ws_l3.data_validations.dataValidation.clear()
    dv_l3 = DataValidation(type="list", formula1='=INDIRECT("L2_" & B1)', showDropDown=True)
    ws_l3.add_data_validation(dv_l3)
    dv_l3.add(ws_l3["B1"])
    ws_l3.merge_cells("B1:C1")

    ws_l4.unmerge_cells("B1:C1")
    ws_l4.data_validations.dataValidation.clear()
    dv_l4 = DataValidation(type="list", formula1='=INDIRECT("L3_" & B1)', showDropDown=True)
    ws_l4.add_data_validation(dv_l4)
    dv_l4.add(ws_l4["B1"])
    ws_l4.merge_cells("B1:C1")

    print(f"✅ Cascading dropdowns updated for: {file_path}")

In [ ]:
from openpyxl.worksheet.datavalidation import DataValidation

def update_l2_l3_l4_dropdown(file_path: str, filename: str, ws_options, ws_l2, ws_l3, ws_l4):
    header_map = {cell.value: cell.column_letter for cell in ws_options[1]}
    col_l2 = header_map.get("Business Service Level L2", "E")
    col_l3 = header_map.get("Business Framework Group", "F")
    col_l4 = header_map.get("Business Framework", "G")

    matched_l3 = []
    matched_l4 = []

    # Collect L3s for selected L2
    for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
        l2_val = row[ord(col_l2) - ord('A')].value
        l3_val = row[ord(col_l3) - ord('A')].value
        if l2_val and str(l2_val).strip() == filename:
            matched_l3.append(str(l3_val).strip())

    # Collect L4s for matched L3s
    for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
        l3_val = row[ord(col_l3) - ord('A')].value
        l4_val = row[ord(col_l4) - ord('A')].value
        if l3_val and str(l3_val).strip() in matched_l3:
            matched_l4.append(str(l4_val).strip())

    if matched_l3 or matched_l4:
        # Clear Options!A3, B3, C3 onwards
        for cell in ws_options["A"][2:]:
            cell.value = None
        for cell in ws_options["B"][2:]:
            cell.value = None
        for cell in ws_options["C"][2:]:
            cell.value = None

        # Populate L2 dropdown values (A)
        for i, val in enumerate(sorted(set([filename])), start=3):
            ws_options[f"A{i}"] = val

        # Populate L3 dropdown values (B)
        for i, val in enumerate(sorted(set(matched_l3)), start=3):
            ws_options[f"B{i}"] = val

        # Populate L4 dropdown values (C)
        for i, val in enumerate(sorted(set(matched_l4)), start=3):
            ws_options[f"C{i}"] = val

        # Update L2 View!B1
        if ws_l2["B1"].coordinate in ws_l2.merged_cells:
            ws_l2.unmerge_cells("B1:C1")
        ws_l2.merge_cells("B1:C1")
        ws_l2["B1"] = filename

        # Update L3 View!B1
        if ws_l3["B1"].coordinate in ws_l3.merged_cells:
            ws_l3.unmerge_cells("B1:C1")
        ws_l3.merge_cells("B1:C1")
        ws_l3["B1"] = filename

        # Update L4 View!B1 with dropdown
        if ws_l4["B1"].coordinate in ws_l4.merged_cells:
            ws_l4.unmerge_cells("B1:C1")
        ws_l4.merge_cells("B1:C1")
        ws_l4.data_validations.dataValidation.clear()

        formula_range = f"=Options!$C$2:$C${len(set(matched_l4))+2}"
        dv = DataValidation(type="list", formula1=formula_range, showDropDown=False)
        dv.error = "Invalid selection."
        dv.prompt = "Choose from dropdown"
        ws_l4.add_data_validation(dv)
        dv.add(ws_l4["B1"])

        print(f"✅ L2 → L3 → L4 dropdowns updated for: {file_path}")
    else:
        print(f"⚠️ No matching values for '{filename}' in {file_path} [L2 Split]")

In [ ]:
print("="*60)
print("📌 Pre-run Checklist for the Split Tool")
print("="*60)
print("1. Ensure the file you are about to split has all sheets properly visible.")
print("2. Make sure **no dropdowns have any value selected** before proceeding.")
print("3. Update column **M** of the 'Options' sheet with the **last date of the month**.")
print("4. Confirm that the 'Options' sheet has **all necessary mappings** in place.")
print("5. Check the 'Data' sheet and **remove any unwanted month-related data**.")
print("\n✅ Once all of the above is done, press Enter to start the tool.")
input()


In [ ]:
import os
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

def update_l3_split_dropdown(file_path: str, filename: str, ws_options, ws_l3, ws_l4):
    header_map = {cell.value: cell.column_letter for cell in ws_options[1]}
    col_l3 = header_map.get("Business Framework Group", "D")
    col_l4 = header_map.get("Business Framework", "E")

    matched_values = []
    for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
        l3_val = row[ord(col_l3) - ord('A')].value
        l4_val = row[ord(col_l4) - ord('A')].value
        if l3_val and str(l3_val).strip() == filename:
            matched_values.append(str(l4_val).strip())

    if matched_values:
        # Clear Options!B3 onwards
        for cell in ws_options["B"][2:]:
            cell.value = None
        for i, val in enumerate(matched_values, start=3):
            ws_options[f"B{i}"] = val

        # Update L3 View!B1 with filename
        if ws_l3["B1"].coordinate in ws_l3.merged_cells:
            ws_l3.unmerge_cells("B1:C1")
        ws_l3.merge_cells("B1:C1")
        ws_l3["B1"] = filename

        # Add dropdown in L4 View!B1
        if ws_l4["B1"].coordinate in ws_l4.merged_cells:
            ws_l4.unmerge_cells("B1:C1")
        ws_l4.merge_cells("B1:C1")
        ws_l4.data_validations.dataValidation.clear()

        formula_range = f"=Options!$B$2:$B${len(matched_values)+2}"
        dv = DataValidation(type="list", formula1=formula_range, showDropDown=False)
        dv.error = "Invalid selection."
        dv.prompt = "Choose from dropdown"
        ws_l4.add_data_validation(dv)
        dv.add(ws_l4["B1"])

        print(f"✅ L3 dropdown updated for: {file_path}")
    else:
        print(f"⚠️ No matching L4 values for '{filename}' in {file_path} [L3 Split]")


def update_region_split_dropdown(file_path: str, filename: str, ws_options, ws_region, ws_country):
    header_map = {cell.value: cell.column_letter for cell in ws_options[1]}
    col_region = header_map.get("Region", "I")
    col_country = header_map.get("Country", "J")

    matched_countries = []
    for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
        region_val = row[ord(col_region) - ord('A')].value
        country_val = row[ord(col_country) - ord('A')].value
        if region_val and str(region_val).strip() == filename:
            matched_countries.append(str(country_val).strip())

    if matched_countries:
        # Clear Options!H3 onwards
        for cell in ws_options["H"][2:]:
            cell.value = None
        for i, val in enumerate(matched_countries, start=3):
            ws_options[f"H{i}"] = val

        # Update Region View!B1 with filename
        if ws_region["B1"].coordinate in ws_region.merged_cells:
            ws_region.unmerge_cells("B1:C1")
        ws_region.merge_cells("B1:C1")
        ws_region["B1"] = filename

        # Add dropdown in Country View!B1 (merged)
        if ws_country["B1"].coordinate in ws_country.merged_cells:
            ws_country.unmerge_cells("B1:C1")
        ws_country.merge_cells("B1:C1")
        ws_country.data_validations.dataValidation.clear()

        formula_range = f"=Options!$H$2:$H${len(matched_countries)+2}"
        dv = DataValidation(type="list", formula1=formula_range, showDropDown=False)
        dv.error = "Invalid selection."
        dv.prompt = "Choose from dropdown"
        ws_country.add_data_validation(dv)
        dv.add(ws_country["B1"])

        print(f"✅ Region dropdown updated for: {file_path}")
    else:
        print(f"⚠️ No matching country values for '{filename}' in {file_path} [Region Split]")


def update_dropdowns_for_folder(folder_path: str):
    for file in os.listdir(folder_path):
        if not file.endswith(".xlsx"):
            continue

        file_path = os.path.join(folder_path, file)
        filename = os.path.splitext(file)[0]

        wb = load_workbook(file_path)
        if "Options" not in wb.sheetnames:
            print(f"Skipping {file} - 'Options' sheet missing.")
            continue

        ws_options = wb["Options"]

        # Handle L3 Split
        if all(sheet in wb.sheetnames for sheet in ["L3 View", "L4 View"]):
            ws_l3 = wb["L3 View"]
            ws_l4 = wb["L4 View"]
            update_l3_split_dropdown(file_path, filename, ws_options, ws_l3, ws_l4)

        # Handle Region Split
        if all(sheet in wb.sheetnames for sheet in ["Region View", "Country View"]):
            ws_region = wb["Region View"]
            ws_country = wb["Country View"]
            update_region_split_dropdown(file_path, filename, ws_options, ws_region, ws_country)

        wb.save(file_path)


In [ ]:
import os
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

def update_l3_type_dropdowns(folder_path: str):
    for file in os.listdir(folder_path):
        if not file.endswith(".xlsx"):
            continue

        file_path = os.path.join(folder_path, file)
        filename = os.path.splitext(file)[0]

        wb = load_workbook(file_path)
        if not all(sheet in wb.sheetnames for sheet in ["Options", "L3 View", "L4 View"]):
            print(f"Skipping {file} - missing required sheets.")
            continue

        ws_options = wb["Options"]
        ws_l3 = wb["L3 View"]
        ws_l4 = wb["L4 View"]

        # Get header map (row 1)
        header_map = {cell.value: cell.column_letter for cell in ws_options[1]}

        col_l3 = header_map.get("Business Framework Group", "D")
        col_l4 = header_map.get("Business Framework", "E")

        if not col_l3 or not col_l4:
            print(f"⚠️ Could not find required headers in 'Options' sheet of {file}")
            continue

        # Collect matched L4 values
        matched_values = []
        for row in ws_options.iter_rows(min_row=2, max_row=ws_options.max_row):
            l3_val = row[ord(col_l3) - ord('A')].value
            l4_val = row[ord(col_l4) - ord('A')].value

            if l3_val and str(l3_val).strip() == filename:
                matched_values.append(str(l4_val).strip())

        if not matched_values:
            print(f"⚠️ No matching L4 values for '{filename}' in {file}")
            continue

        # Clear old values in Options!B3 onwards
        for cell in ws_options["B"][2:]:
            cell.value = None

        # Paste new values into Options!B3 onward (keeping B2 for '*')
        for i, val in enumerate(matched_values, start=3):
            ws_options[f"B{i}"] = val

        # Update L3 View B1 with filename (merge B1:C1 if needed)
        try:
            ws_l3.unmerge_cells("B1:C1")
        except:
            pass
        ws_l3.merge_cells("B1:C1")
        ws_l3["B1"] = filename

        # Prepare L4 View B1:C1 merged cell
        try:
            ws_l4.unmerge_cells("B1:C1")
        except:
            pass
        ws_l4.merge_cells("B1:C1")

        # Clear existing validations
        ws_l4.data_validations.dataValidation.clear()

        # Add dropdown (include B2 where '*' is already present)
        total_rows = len(matched_values) + 1
        formula_range = f"=Options!$B$2:$B${total_rows + 1}"

        dv = DataValidation(type="list", formula1=formula_range, showDropDown=False)
        dv.error = "Invalid selection."
        dv.prompt = "Choose from dropdown"

        ws_l4.add_data_validation(dv)
        dv.add(ws_l4["B1"])  # Only top-left of merged cell

        wb.save(file_path)
        print(f"✅ Dropdown updated for: {file}")


In [ ]:
import os
import openpyxl
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter

def update_dropdown_for_file(
    file_path,
    source_sheet, key_col, value_col,
    dropdown_source_col,
    key_sheet_name, key_cell_to_update, clear_cell,
    dropdown_sheet_name, dropdown_cell
):
    wb = openpyxl.load_workbook(file_path)
    ws_source = wb[source_sheet]
    ws_key = wb[key_sheet_name]
    ws_dropdown = wb[dropdown_sheet_name]

    # Get key from filename (without extension)
    key_value = os.path.splitext(os.path.basename(file_path))[0]

    # Step 1: Build dictionary from key_col and value_col
    value_dict = {}
    for row in range(2, ws_source.max_row + 1):
        key = ws_source.cell(row=row, column=key_col).value
        val = ws_source.cell(row=row, column=value_col).value
        if key and val:
            if key not in value_dict:
                value_dict[key] = set()
            value_dict[key].add(val)

    # Step 2: Insert "*" as first value for each key
    for k in value_dict:
        value_dict[k] = ["*"] + sorted(value_dict[k])

    # Step 3: Update key view
    ws_key[key_cell_to_update] = key_value
    ws_key[clear_cell] = ""

    # Step 4: Clear previous dropdown source values
    for row in range(3, ws_source.max_row + 1):
        ws_source.cell(row=row, column=dropdown_source_col).value = ""

    # Step 5: Write new dropdown values from dict
    values = value_dict.get(key_value, ["*"])
    for i, val in enumerate(values, start=3):
        ws_source.cell(row=i, column=dropdown_source_col).value = val

    # Step 6: Clear old validations
    ws_dropdown.data_validations.clear()

    # Step 7: Create new validation formula
    col_letter = get_column_letter(dropdown_source_col)
    formula = f"=OFFSET({source_sheet}!${col_letter}$3,0,0,COUNTA({source_sheet}!${col_letter}$3:${col_letter}$1000),1)"
    dv = DataValidation(type="list", formula1=formula, showDropDown=True)
    ws_dropdown.add_data_validation(dv)
    dv.add(ws_dropdown[dropdown_cell])

    # Step 8: Set default dropdown value
    ws_dropdown[dropdown_cell] = "*"

    wb.save(file_path)
    print(f"Updated dropdowns in {os.path.basename(file_path)}")


def update_dropdowns_l3(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith(".xlsm"):
            update_dropdown_for_file(
                file_path=os.path.join(folder_path, file),
                source_sheet="Options",
                key_col=4, value_col=5,          # D & E
                dropdown_source_col=2,           # B
                key_sheet_name="L3 View",
                key_cell_to_update="B1",
                clear_cell="E1",
                dropdown_sheet_name="L4 View",
                dropdown_cell="B1"
            )

def update_dropdowns_region(folder_path):
    for file in os.listdir(folder_path):
        if file.endswith(".xlsm"):
            update_dropdown_for_file(
                file_path=os.path.join(folder_path, file),
                source_sheet="Options",
                key_col=9, value_col=10,         # I & J
                dropdown_source_col=2,           # B
                key_sheet_name="Region View",
                key_cell_to_update="B1",
                clear_cell="E1",
                dropdown_sheet_name="Country View",
                dropdown_cell="B1"
            )
-
update_dropdowns_l3("Filtered_files_l3_wise")
update_dropdowns_region("Filtered_files_region_wise")
update_dropdowns_l3(output_folder_l3)


In [1]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Gift Card.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Credit Card.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\ Cash.xlsx


In [3]:
# import tkinter as tk
# from tkinter import filedialog, messagebox, simpledialog, ttk
# temp_root = tk.Tk()
# temp_root.withdraw()
# messagebox.showerror(
#     "Error", 
#     "❌ You selected all sheets to exclude, including the one you want to split.\n\n"
#     "Exiting... Please re-run the tool with proper selections.",
#     parent=temp_root
# )
# temp_root.destroy()

In [1]:
# 22-4-2025 ---final 
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"
def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")


def show_error_gracefully(title, message):
    temp_root = tk.Tk()
    temp_root.withdraw()
    messagebox.showerror(title, message, parent=temp_root)
    temp_root.destroy()


    
def show_welcome():
#     def on_close():
#         # Ask the user if they want to exit when they click the cross
#         if messagebox.askyesno("Exit", "Do you want to exit the application?"):
#             root.quit()  # Exit the program
#             root.destroy()  # Properly clean up and destroy the window
#         else:
#             pass  # If user chooses "No", we simply return and continue running  
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x400")
#     root.protocol("WM_DELETE_WINDOW", on_close)
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    
#   # After closing, proceed to the next step
#     file_path = select_file()
#     if not file_path:
#         return
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()
    def on_closing():
        result["value"] = None
        root.quit()
        
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x400")
    root.protocol("WM_DELETE_WINDOW", on_closing)

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()
    def on_closing():
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    root.protocol("WM_DELETE_WINDOW", on_closing)
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    
    root = tk.Tk()
    def on_closing():
        root.quit()
    root.attributes("-topmost", True)
    root.withdraw()
    root.protocol("WM_DELETE_WINDOW", on_closing)
    
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    if row is None:
        root.quit()
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}",
            parent=temp_root
            )
        temp_root.destroy()
        
        
        if sheet_to_split in exclude_sheets:
            temp_root = tk.Tk()
            temp_root.withdraw()
#             messagebox.showinfo("Error", "excluded all sheets.")
            print("exiting")

#         if tk._default_root:
# #             for widget in tk._default_root.winfo_children():
# #                 widget.destroy()
# #             tk._default_root.destroy()
            messagebox.showerror("Error", 
                "❌ You selected all sheets to exclude, including the one you want to split.\n\n"
                "Exiting... Please re-run the tool with proper selections.")

            if tk._default_root:
                for widget in tk._default_root.winfo_children():
                    widget.destroy()
                tk._default_root.destroy()
            return


        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        print(output_dir)
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}",)
        print("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Error", "excluded all sheets.",e)
        print("Exiting....")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [6]:
## handle close
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"
def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")


def show_error_gracefully(title, message):
    temp_root = tk.Tk()
    temp_root.withdraw()
    messagebox.showerror(title, message, parent=temp_root)
    temp_root.destroy()

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x400")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    
    # Proper close handling for the start window
    def on_close_start():
        if messagebox.askyesno("Exit", "Do you want to exit the application?"):
            root.quit()
            root.destroy()
    
    root.protocol("WM_DELETE_WINDOW", on_close_start)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    def on_closing():
        result["value"] = None
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x400")
    root.protocol("WM_DELETE_WINDOW", on_closing)

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    
    # Proper close handling for dropdown window
    def on_close_dropdown():
        result["value"] = None
        root.quit()
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_close_dropdown)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    def on_closing():
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    root.protocol("WM_DELETE_WINDOW", on_closing)
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    
    # Proper close handling for multi-select window
    def on_close_multi_select():
        root.quit()
        root.destroy()

    root.protocol("WM_DELETE_WINDOW", on_close_multi_select)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    root = tk.Tk()
    def on_closing():
        root.quit()

    root.attributes("-topmost", True)
    root.withdraw()
    root.protocol("WM_DELETE_WINDOW", on_closing)
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    if row is None:
        root.quit()
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}",
            parent=temp_root
            )
        temp_root.destroy()
        
        if sheet_to_split in exclude_sheets:
            temp_root = tk.Tk()
            temp_root.withdraw()
            print("exiting")
            messagebox.showerror("Error", 
                "❌ You selected all sheets to exclude, including the one you want to split.\n\n"
                "Exiting... Please re-run the tool with proper selections.")

            if tk._default_root:
                for widget in tk._default_root.winfo_children():
                    widget.destroy()
                tk._default_root.destroy()
            return


        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}",)
        print(f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Error", "Error during file processing.",e)
        print("Exiting....")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


TypeError: showinfo() takes from 0 to 2 positional arguments but 3 were given

In [2]:
## wizard
# 22-4-2025
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"
def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")


class WizardApp:
    def __init__(self):
        self.file_path = None
        self.sheet_names = []
        self.sheet_to_split = None
        self.start_row = 1
        self.split_column = None
        self.exclude_sheets = []
        self.df = None

        self.root = tk.Tk()
        self.root.title("Excel Splitter Wizard")
        self.root.geometry("500x300")
        self.current_step = 0
        self.frames = []

        self.steps = [
            self.welcome_step,
            self.file_selection_step,
            self.sheet_selection_step,
            self.row_selection_step,
            self.column_selection_step,
            self.exclude_sheet_step,
            self.confirm_step
        ]

        self.create_nav_buttons()
        self.show_step(0)
        self.root.mainloop()

    def create_nav_buttons(self):
        nav_frame = tk.Frame(self.root)
        nav_frame.pack(side=tk.BOTTOM, fill=tk.X)
        self.back_btn = ttk.Button(nav_frame, text="Back", command=self.back)
        self.back_btn.pack(side=tk.LEFT, padx=10, pady=5)
        self.next_btn = ttk.Button(nav_frame, text="Next", command=self.next)
        self.next_btn.pack(side=tk.RIGHT, padx=10, pady=5)

    def clear_window(self):
        for frame in self.frames:
            frame.pack_forget()
            frame.destroy()
        self.frames.clear()

    def show_step(self, step):
        self.clear_window()
        self.current_step = step
        self.steps[step]()
        self.back_btn.config(state=tk.NORMAL if step > 0 else tk.DISABLED)
        self.next_btn.config(text="Next" if step < len(self.steps)-1 else "Finish")

    def back(self):
        if self.current_step > 0:
            self.show_step(self.current_step - 1)

    def next(self):
        if self.current_step < len(self.steps) - 1:
            self.show_step(self.current_step + 1)
        else:
            self.root.destroy()
            split_excel_preserving_format_and_formulas(self)

    def welcome_step(self):
        frame = tk.Frame(self.root)
        ttk.Label(frame, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
        ttk.Label(frame, text="Split a sheet by column and preserve formatting.").pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def file_selection_step(self):
        def browse():
            path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
            if path:
                self.file_path = path
                try:
                    xls = pd.ExcelFile(path)
                    self.sheet_names = xls.sheet_names
                    self.df = pd.read_excel(path, sheet_name=0)
                    label_var.set(os.path.basename(path))
                except Exception as e:
                    messagebox.showerror("Error", f"Failed to read file: {e}")

        frame = tk.Frame(self.root)
        label_var = tk.StringVar(value="No file selected")
        ttk.Label(frame, text="Select Excel File").pack(pady=5)
        ttk.Button(frame, text="Browse", command=browse).pack(pady=5)
        ttk.Label(frame, textvariable=label_var).pack(pady=5)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def sheet_selection_step(self):
        frame = tk.Frame(self.root)
        ttk.Label(frame, text="Select sheet to split").pack(pady=10)
        combo = ttk.Combobox(frame, values=self.sheet_names, state="readonly")
        combo.pack(pady=5)
        if self.sheet_to_split:
            combo.set(self.sheet_to_split)
        def update():
            self.sheet_to_split = combo.get()
            self.df = pd.read_excel(self.file_path, sheet_name=self.sheet_to_split)
        ttk.Button(frame, text="Confirm", command=update).pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def row_selection_step(self):
        frame = tk.Frame(self.root)
        ttk.Label(frame, text="Enter data start row").pack(pady=10)
        row_var = tk.IntVar(value=self.start_row)
        spin = tk.Spinbox(frame, from_=1, to=100, textvariable=row_var)
        spin.pack(pady=5)
        def update():
            self.start_row = row_var.get()
        ttk.Button(frame, text="Confirm", command=update).pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def column_selection_step(self):
        frame = tk.Frame(self.root)
        ttk.Label(frame, text="Select column to split by").pack(pady=10)
        combo = ttk.Combobox(frame, values=list(self.df.columns), state="readonly")
        combo.pack(pady=5)
        if self.split_column:
            combo.set(self.split_column)
        def update():
            self.split_column = combo.get()
        ttk.Button(frame, text="Confirm", command=update).pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def exclude_sheet_step(self):
        frame = tk.Frame(self.root)
        ttk.Label(frame, text="Select sheets to exclude (optional)").pack(pady=10)
        checks = []
        vars = []
        for name in self.sheet_names:
            var = tk.BooleanVar(value=name in self.exclude_sheets)
            chk = tk.Checkbutton(frame, text=name, variable=var)
            chk.pack(anchor="w")
            checks.append(chk)
            vars.append((name, var))

        def update():
            self.exclude_sheets = [name for name, var in vars if var.get()]
        ttk.Button(frame, text="Confirm", command=update).pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)

    def confirm_step(self):
        frame = tk.Frame(self.root)
        info = (
            f"Sheet to split: {self.sheet_to_split}\n"
            f"Split by column: {self.split_column}\n"
            f"Data starts from row: {self.start_row}\n"
            f"Exclude sheets: {', '.join(self.exclude_sheets) if self.exclude_sheets else 'None'}"
        )
        ttk.Label(frame, text="Confirm your selections", font=("Segoe UI", 12, "bold")).pack(pady=10)
        ttk.Label(frame, text=info).pack(pady=10)
        frame.pack(fill=tk.BOTH, expand=True)
        self.frames.append(frame)


def split_excel_preserving_format_and_formulas(wizard):
    file_path = wizard.file_path
    sheet_to_split = wizard.sheet_to_split
    start_row = wizard.start_row
    split_column = wizard.split_column
    exclude_sheets = wizard.exclude_sheets
    df = wizard.df
    
    if not all([file_path, sheet_to_split, start_row, split_column]):
        messagebox.showerror("Error", "Missing required selections. Please restart the wizard.")
        return

    try:
        if sheet_to_split in exclude_sheets:
            messagebox.showerror("Error", "Selected sheet to split is also in exclude list.")
            return

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
        messagebox.showinfo("Done", f"Files split and saved to: {output_dir}")

    except Exception as e:
        messagebox.showerror("Error", f"Something went wrong: {e}")


if __name__ == "__main__":
    WizardApp()


In [5]:
# final run
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()  # ✅ Ensure window is closed

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()  # ✅ Fully close the window
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Cleanup after loop ends
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()  # ✅ Fully close after use
    return result

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set as the only selected tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Write filtered data
                filtered_df = df[df[split_column] == val]
                for i, row in enumerate(filtered_df.values, start=2):
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=i, column=j).value = value

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
#                 messagebox.showinfo("Done", f"Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Force close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


✅ Saved: C:/Users/KS\Split_Formatted_Excel\1.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\2.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\3.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\4.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\5.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\6.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\7.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\8.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\9.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\10.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\11.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\12.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\13.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\14.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\15.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\16.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\17.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\18.xlsx
✅ Saved: C:/Users/KS\Split_Formatted_Excel\19.xlsx


KeyboardInterrupt: 

In [1]:
# format preservation
# final run with formatting & formula preservation
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

# 👉 Helper to copy formatting & formula
def copy_row(ws_from, ws_to, row_idx_from, row_idx_to):
    for col_idx, cell in enumerate(ws_from[row_idx_from], start=1):
        new_cell = ws_to.cell(row=row_idx_to, column=col_idx)
        new_cell.value = cell.value
        new_cell.data_type = cell.data_type

        if cell.has_style:
            new_cell.font = copy(cell.font)
            new_cell.border = copy(cell.border)
            new_cell.fill = copy(cell.fill)
            new_cell.number_format = copy(cell.number_format)
            new_cell.protection = copy(cell.protection)
            new_cell.alignment = copy(cell.alignment)

        if cell.hyperlink:
            new_cell.hyperlink = copy(cell.hyperlink)

        if cell.comment:
            new_cell.comment = copy(cell.comment)

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]

            # Set as active tab
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            # Copy original sheet (used to preserve formulas/formatting)
            original_ws = load_workbook(file_path, data_only=False)[sheet_to_split]

            # Clear old data below header
            max_row = ws.max_row
            if max_row > 1:
                ws.delete_rows(2, max_row - 1)

            filtered_df = df[df[split_column] == val]

            for idx, (_, row) in enumerate(filtered_df.iterrows(), start=2):
                source_excel_row_index = row.name + 2  # Adjust for Excel's row numbers
                copy_row(original_ws, ws, source_excel_row_index, idx)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


UnicodeEncodeError: 'charmap' codec can't encode character '\u274c' in position 0: character maps to <undefined>

In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.worksheet import Worksheet

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def copy_row(source_ws: Worksheet, dest_ws: Worksheet, source_row: int, dest_row: int):
    for col in range(1, source_ws.max_column + 1):
        source_cell = source_ws.cell(row=source_row, column=col)
        dest_cell = dest_ws.cell(row=dest_row, column=col)

        dest_cell.value = source_cell.value
        dest_cell.data_type = source_cell.data_type
        dest_cell.number_format = source_cell.number_format
        dest_cell.font = source_cell.font
        dest_cell.border = source_cell.border
        dest_cell.fill = source_cell.fill
        dest_cell.alignment = source_cell.alignment

        if source_cell.has_formula:
            dest_cell.value = f"={source_cell.value}"

def split_excel_preserving_format():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            original_wb = load_workbook(file_path)
            original_ws = original_wb[sheet_to_split]

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Set tab active
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Clear previous data except header
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filtered data
                filtered_df = df[df[split_column] == val]

                # ✅ Copy matching rows with formatting and formulas
                for i in range(len(filtered_df)):
                    excel_source_row_index = filtered_df.index[i] + 2  # original Excel row (1-based + header)
                    target_row_index = i + 2  # destination row in new sheet
                    copy_row(original_ws, ws, excel_source_row_index, target_row_index)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")

        # Close any leftover windows
        for widget in tk._default_root.winfo_children():
            widget.destroy()
        if tk._default_root:
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xlsm")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_everything():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, dtype=str)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            if sheet_to_split not in wb.sheetnames:
                continue

            # Remove excluded sheets
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            ws = wb[sheet_to_split]
            headers = [cell.value for cell in ws[1]]

            # Create a mapping of column name to column number
            col_map = {cell.value: idx+1 for idx, cell in enumerate(ws[1])}
            filter_col_idx = col_map.get(split_column)
            if not filter_col_idx:
                continue

            rows_to_keep = [1]  # Always keep header
            for i in range(2, ws.max_row + 1):
                cell_val = ws.cell(i, filter_col_idx).value
                if str(cell_val).strip() == str(val).strip():
                    rows_to_keep.append(i)

            # Delete rows not in the filtered list
            all_rows = list(range(2, ws.max_row + 1))
            delete_rows = [i for i in all_rows if i not in rows_to_keep]

            for row in reversed(delete_rows):
                ws.delete_rows(row, 1)

            # Ensure it's the active selected sheet
            for w in wb.worksheets:
                w.sheet_view.tabSelected = False
            ws.sheet_view.tabSelected = True
            wb.active = wb.sheetnames.index(sheet_to_split)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split successfully!\nSaved to:\n{output_dir}")

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_everything()


In [ ]:
#  Save formulas too
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup formulas from first data row
                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                # Clear previous data rows
                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    # Apply formulas row-wise
                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# Final Code with formatting 

import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from tkinter import simpledialog  # Add this if you need askstring
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=1"0)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("300x150")
    root.attributes("-topmost", True)

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.title(title)
    root.geometry("350x450")
    root.attributes("-topmost", True)

    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()
    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                start_row = 2
                formulas = []
                for cell in ws[start_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > 1:
                    ws.delete_rows(2, max_row - 1)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", msg)
        log_error(msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# perfect just adjust data start row

import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
from copy import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=10)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Select this tab
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                # Backup template row 2
                start_row = 2
                template_row = list(ws.iter_rows(min_row=start_row, max_row=start_row))[0]
                template_data = []
                for cell in template_row:
                    template_data.append({
                        "value": cell.value,
                        "is_formula": cell.data_type == "f",
                        "style": copy(cell._style),
                        "number_format": cell.number_format
                    })

                # Clear all data rows after header
                if ws.max_row > 1:
                    ws.delete_rows(start_row, ws.max_row - 1)

                # Filter data
                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, (value, cell_template) in enumerate(zip(row, template_data), start=1):
                        cell = ws.cell(row=start_row + i, column=j)
                        cell.value = value
                        cell._style = copy(cell_template["style"])
                        cell.number_format = cell_template["number_format"]

                    for j, cell_template in enumerate(template_data, start=1):
                        if cell_template["is_formula"]:
                            origin = f"{get_column_letter(j)}{start_row}"
                            target = f"{get_column_letter(j)}{start_row + i}"
                            translated_formula = Translator("=" + str(cell_template["value"]), origin=origin).translate_formula(target)
                            cell = ws.cell(row=start_row + i, column=j)
                            cell.value = translated_formula
                            cell._style = copy(cell_template["style"])
                            cell.number_format = cell_template["number_format"]

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split with formatting + formulas!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()
    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import pandas as pd
import xlwings as xw
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x250")

    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=10)

    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Select an Excel file",
        filetypes=[("Excel files", "*.xlsx *.xls")]
    )
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}

    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x200")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []

    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result

def ask_start_row():
    result = {"value": 1}

    def confirm():
        try:
            row_val = int(entry.get())
            if row_val >= 1:
                result["value"] = row_val
        except:
            pass
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("Data Start Row")
    root.geometry("300x150")

    ttk.Label(root, text="Enter the row number where data headers start:").pack(pady=10)
    entry = ttk.Entry(root)
    entry.insert(0, "1")
    entry.pack(pady=5)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)

    root.mainloop()
    root.destroy()
    return result["value"]

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = ask_start_row()

        # Use xlwings to fetch header row
        app_temp = xw.App(visible=False)
        wb_temp = app_temp.books.open(file_path)
        ws_temp = wb_temp.sheets[sheet_to_split]
        header_row = ws_temp.range((start_row, 1), (start_row, ws_temp.used_range.last_cell.column)).value
        wb_temp.close()
        app_temp.quit()

        split_column = get_dropdown_selection(header_row, "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=None)
        df.columns = df.iloc[start_row - 1]
        df = df.iloc[start_row:]

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                start_excel_row = start_row + 1
                formulas = []
                for cell in ws[start_excel_row]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_excel_row:
                    ws.delete_rows(start_excel_row + 1, max_row - start_excel_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_excel_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_excel_row}"
                            target_cell = f"{get_column_letter(j)}{start_excel_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_excel_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        subprocess.Popen(f'explorer "{output_dir}"')
        messagebox.showinfo("Done", f"🎉 Files split and formatting + formulas preserved!\nSaved to:\n{output_dir}")
        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
# 18-4-2025

In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                for cell in ws[excel_data_start]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)  # Keep header

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i  # Write below header
                        ws.cell(row=cell_row, column=j, value=value)

                        # Reapply formulas if applicable
                        orig_formula = formulas[j - 1]
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=cell_row, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formulas preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
import shutil
import tempfile
import subprocess
import datetime

def log_error(msg):
    print("Error:", msg)

def get_selected_items(listbox, items):
    selected_indices = listbox.curselection()
    return [items[i] for i in selected_indices]

def main():
    root = tk.Tk()
    root.withdraw()

    try:
        # Step 1: Welcome message
        welcome = tk.Toplevel()
        welcome.title("Welcome")
        welcome.geometry("400x150")
        tk.Label(welcome, text="📂 Excel Splitter Tool", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Label(welcome, text="This tool will split your Excel sheet based on selected values.").pack()
        tk.Button(welcome, text="Get Started", command=welcome.destroy).pack(pady=10)
        welcome.grab_set()
        welcome.mainloop()

        # Step 2: Ask user to select file
        file_path = filedialog.askopenfilename(
            title="Select Excel file",
            filetypes=[("Excel Files", "*.xlsx *.xls")]
        )
        if not file_path:
            messagebox.showinfo("No file selected", "Operation cancelled.")
            return

        # Step 3: Load sheet names
        wb = load_workbook(file_path, data_only=True)
        sheet_names = wb.sheetnames

        # Step 4: Sheet selection window
        sheet_win = tk.Toplevel()
        sheet_win.title("Select Sheet")
        sheet_win.geometry("300x150")
        tk.Label(sheet_win, text="Select sheet to split:").pack(pady=5)
        selected_sheet = tk.StringVar(value=sheet_names[0])
        sheet_dropdown = ttk.Combobox(sheet_win, textvariable=selected_sheet, values=sheet_names)
        sheet_dropdown.pack(pady=10)

        def confirm_sheet():
            if selected_sheet.get() not in sheet_names:
                messagebox.showerror("Invalid", "Please select a valid sheet.")
            else:
                sheet_win.destroy()

        tk.Button(sheet_win, text="Next", command=confirm_sheet).pack(pady=10)
        sheet_win.grab_set()
        sheet_win.mainloop()

        # Step 5: Ask for header start row
        header_row = simpledialog.askinteger("Header Row", "Enter row number where header starts:")
        if not header_row:
            messagebox.showerror("Invalid Input", "Header row not specified.")
            return

        # Load selected sheet into DataFrame
        df = pd.read_excel(file_path, sheet_name=selected_sheet.get(), header=header_row - 1)
        column_choices = list(df.columns)

        # Step 6: Ask user to select the column to split by
        col_win = tk.Toplevel()
        col_win.title("Select Split Column")
        col_win.geometry("400x150")
        tk.Label(col_win, text="Select column to split by:").pack(pady=5)
        selected_col = tk.StringVar(value=column_choices[0])
        col_dropdown = ttk.Combobox(col_win, textvariable=selected_col, values=column_choices)
        col_dropdown.pack(pady=10)

        def confirm_col():
            if selected_col.get() not in column_choices:
                messagebox.showerror("Invalid", "Please select a valid column.")
            else:
                col_win.destroy()

        tk.Button(col_win, text="Next", command=confirm_col).pack(pady=10)
        col_win.grab_set()
        col_win.mainloop()

        split_column = selected_col.get()

        # Step 7: Ask user to select which values to split
        unique_vals = df[split_column].dropna().unique().tolist()

        select_win = tk.Toplevel()
        select_win.title("Select Values to Split")
        select_win.geometry("400x400")
        tk.Label(select_win, text="Select values to create individual files:").pack(pady=5)

        listbox = tk.Listbox(select_win, selectmode=tk.MULTIPLE)
        for val in unique_vals:
            listbox.insert(tk.END, str(val))
        listbox.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)

        selected_vals = []

        def confirm_values():
            selected_vals.extend(get_selected_items(listbox, unique_vals))
            if not selected_vals:
                messagebox.showerror("Error", "Please select at least one value.")
                return
            select_win.destroy()

        tk.Button(select_win, text="Split Files", command=confirm_values).pack(pady=10)
        select_win.grab_set()
        select_win.mainloop()

        if not selected_vals:
            return

        # Step 8: Ask where to save the split files
        output_dir = filedialog.askdirectory(title="Select folder to save split files")
        if not output_dir:
            messagebox.showinfo("No folder", "No folder selected. Exiting.")
            return

        # Step 9: Split and save
        for val in selected_vals:
            temp_dir = tempfile.mkdtemp()
            temp_file = os.path.join(temp_dir, "temp.xlsx")
            shutil.copy(file_path, temp_file)

            wb = load_workbook(temp_file)
            if selected_sheet.get() in wb.sheetnames:
                ws = wb[selected_sheet.get()]
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(selected_sheet.get())

                col_index = df.columns.get_loc(split_column) + 1  # 1-based index

                rows_to_delete = []
                for row in range(header_row + 1, ws.max_row + 1):
                    cell_value = ws.cell(row=row, column=col_index).value
                    if cell_value != val:
                        rows_to_delete.append(row)

                for row in reversed(rows_to_delete):
                    ws.delete_rows(row)

                # Reinstate header (if needed)
                for col_num, column_name in enumerate(df.columns, 1):
                    ws.cell(row=header_row, column=col_num, value=column_name)

                safe_val = str(val).replace("/", "_").replace("\\", "_").replace(":", "_")
                timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                new_file = os.path.join(output_dir, f"{safe_val}_{timestamp}.xlsx")
                wb.save(new_file)

            shutil.rmtree(temp_dir)

        # Final message
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

    finally:
        if tk._default_root:
            try:
                for widget in tk._default_root.winfo_children():
                    widget.destroy()
                tk._default_root.destroy()
            except:
                pass

if __name__ == "__main__":
    main()


In [ ]:
## ok


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 2):", initialvalue=1)
    root.destroy()
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_welcome():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x200")
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()
    root.destroy()

def select_file():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x150")

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()
    return result

def get_data_start_row():
    root = tk.Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    root.destroy()
    return row

def show_processing_window():
    processing_root = tk.Tk()
    processing_root.title("Processing")
    processing_root.geometry("400x100")
    processing_root.resizable(False, False)
    processing_root.attributes("-topmost", True)
    ttk.Label(processing_root, text="📂 Splitter Tool is working on files...", font=("Segoe UI", 12)).pack(expand=True)
    processing_root.update()
    return processing_root

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")

        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}"
        )

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        os.makedirs(output_dir, exist_ok=True)

        processing_window = show_processing_window()

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                for cell in ws[excel_data_start]:
                    if cell.data_type == 'f':
                        formulas.append(f"={cell.value}")
                    else:
                        formulas.append(cell.value)

                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        ws.cell(row=start_row + i, column=j, value=value)

                    for j, orig_formula in enumerate(formulas, start=1):
                        if isinstance(orig_formula, str) and orig_formula.startswith("="):
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{start_row + i}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            ws.cell(row=start_row + i, column=j, value=translated)

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")

        processing_window.destroy()
        messagebox.showinfo("Done", f"🎉 Files split and formulas preserved!\nSaved to:\n{output_dir}")
        subprocess.Popen(f'explorer "{output_dir}"')

        if tk._default_root:
            for widget in tk._default_root.winfo_children():
                widget.destroy()
            tk._default_root.destroy()

    except Exception as e:
        error_msg = f"❌ Unexpected error: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()


In [7]:
import os
import shutil
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog, ttk
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Font, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formula.translate import Translator
import subprocess
import copy

LOG_FILE = "error_log.txt"

def log_error(msg):
    with open(LOG_FILE, "a") as f:
        f.write(msg + "\n")

def show_error_gracefully(title, message):
    temp_root = tk.Tk()
    temp_root.withdraw()
    messagebox.showerror(title, message, parent=temp_root)
    temp_root.destroy()

def show_welcome():
    def on_close():
        # Gracefully handle close event without throwing errors
        root.quit()
        root.destroy()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title("🛠 Excel Splitter Tool")
    root.geometry("400x400")
    root.protocol("WM_DELETE_WINDOW", on_close)  # Handle the close button event
    ttk.Label(root, text="Welcome to Excel Splitter Tool", font=("Segoe UI", 14, "bold")).pack(pady=20)
    ttk.Label(root, text="➡ Select a sheet\n➡ Choose a column to split\n➡ Get formatted files", font=("Segoe UI", 10)).pack(pady=10)
    ttk.Button(root, text="Start", command=root.quit).pack(pady=30)
    root.mainloop()

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide root window
    file_path = filedialog.askopenfilename(title="Select an Excel file", filetypes=[("Excel files", "*.xlsx *.xls")])
    root.destroy()  # Properly destroy the tkinter root window after selection
    return file_path

def get_dropdown_selection(options, title):
    result = {}
    def confirm():
        result["value"] = selected_value.get()
        root.quit()

    def on_closing():
        result["value"] = None
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("300x400")
    root.protocol("WM_DELETE_WINDOW", on_closing)  # Handle close button event

    selected_value = tk.StringVar(value=options[0])
    ttk.Label(root, text=title).pack(pady=10)
    dropdown = ttk.Combobox(root, textvariable=selected_value, values=options, state="readonly")
    dropdown.pack(pady=10)
    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result.get("value")

def get_multi_select(options, title):
    result = []
    def confirm():
        for i, var in enumerate(var_list):
            if var.get():
                result.append(options[i])
        root.quit()

    def on_closing():
        root.quit()

    root = tk.Tk()
    root.attributes("-topmost", True)
    root.title(title)
    root.geometry("350x450")
    root.protocol("WM_DELETE_WINDOW", on_closing)  # Handle close button event
    ttk.Label(root, text=title).pack(pady=10)

    var_list = []
    for option in options:
        var = tk.BooleanVar()
        chk = tk.Checkbutton(root, text=option, variable=var)
        chk.pack(anchor='w')
        var_list.append(var)

    ttk.Button(root, text="OK", command=confirm).pack(pady=10)
    root.mainloop()
    root.destroy()  # Properly destroy root window after selection
    return result

def get_data_start_row():
    root = tk.Tk()
    def on_closing():
        root.quit()

    root.attributes("-topmost", True)
    root.withdraw()
    root.protocol("WM_DELETE_WINDOW", on_closing)  # Handle close button event

    row = simpledialog.askinteger("Data Start Row", "Enter the row number where data starts (e.g., 3):", initialvalue=1)
    if row is None:
        root.quit()
    root.destroy()  # Properly destroy root window after getting input
    return row

def copy_cell_style(source_cell, target_cell):
    if source_cell.has_style:
        target_cell.font = copy.copy(source_cell.font)
        target_cell.border = copy.copy(source_cell.border)
        target_cell.fill = copy.copy(source_cell.fill)
        target_cell.number_format = copy.copy(source_cell.number_format)
        target_cell.protection = copy.copy(source_cell.protection)
        target_cell.alignment = copy.copy(source_cell.alignment)

def split_excel_preserving_format_and_formulas():
    show_welcome()

    file_path = select_file()
    if not file_path:
        return

    try:
        xls = pd.ExcelFile(file_path)
        sheet_names = xls.sheet_names
    except Exception as e:
        error_msg = f"❌ Failed to load workbook: {e}"
        messagebox.showerror("Error", error_msg)
        log_error(error_msg)
        return

    try:
        sheet_to_split = get_dropdown_selection(sheet_names, "Select the sheet to split")
        if not sheet_to_split:
            return

        start_row = get_data_start_row()
        if not start_row or start_row < 1:
            messagebox.showerror("Error", "Invalid start row number.")
            return

        df = pd.read_excel(file_path, sheet_name=sheet_to_split, header=start_row - 1)
        split_column = get_dropdown_selection(df.columns.tolist(), "Select column to split by")
        if not split_column:
            return

        exclude_sheets = get_multi_select(sheet_names, "Select sheets to exclude (optional)")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Selections",
            f"✔️ Sheet to split: {sheet_to_split}\n"
            f"✔️ Column to split by: {split_column}\n"
            f"✔️ Data starts from row: {start_row}\n"
            f"✔️ Sheets to exclude: {', '.join(exclude_sheets) if exclude_sheets else 'None'}",
            parent=temp_root
            )
        temp_root.destroy()

        if sheet_to_split in exclude_sheets:
            temp_root = tk.Tk()
            temp_root.withdraw()
            messagebox.showinfo("Error", "You selected all sheets to exclude, including the one you want to split.")
            temp_root.destroy()
            return

        output_dir = os.path.join(os.path.dirname(file_path), "Split_Formatted_Excel")
        print(output_dir)
        os.makedirs(output_dir, exist_ok=True)

        unique_vals = df[split_column].dropna().unique()

        for val in unique_vals:
            dest_file = os.path.join(output_dir, f"{val}.xlsx")
            shutil.copy(file_path, dest_file)

            wb = load_workbook(dest_file)
            for sheet in exclude_sheets:
                if sheet in wb.sheetnames:
                    del wb[sheet]

            if sheet_to_split in wb.sheetnames:
                ws = wb[sheet_to_split]

                # Activate the split sheet
                for w in wb.worksheets:
                    w.sheet_view.tabSelected = False
                ws.sheet_view.tabSelected = True
                wb.active = wb.sheetnames.index(sheet_to_split)

                excel_data_start = start_row + 1  # Excel is 1-indexed
                formulas = []
                styles = []
                for cell in ws[excel_data_start]:
                    formulas.append(f"={cell.value}" if cell.data_type == 'f' else None)
                    styles.append(copy.copy(cell._style))

                # Save column widths
                col_widths = {col: ws.column_dimensions[col].width for col in ws.column_dimensions}

                # Delete data rows only, not headers
                max_row = ws.max_row
                if max_row > start_row:
                    ws.delete_rows(start_row + 1, max_row - start_row)

                filtered_df = df[df[split_column] == val].reset_index(drop=True)

                for i, row in filtered_df.iterrows():
                    for j, value in enumerate(row, start=1):
                        cell_row = start_row + 1 + i
                        cell = ws.cell(row=cell_row, column=j, value=value)

                        # Apply styles
                        if j <= len(styles):
                            cell._style = copy.copy(styles[j - 1])

                        # Translate and reapply formula if needed
                        orig_formula = formulas[j - 1] if j - 1 < len(formulas) else None
                        if orig_formula:
                            origin_cell = f"{get_column_letter(j)}{start_row + 1}"
                            target_cell = f"{get_column_letter(j)}{cell_row}"
                            translated = Translator(orig_formula, origin=origin_cell).translate_formula(target_cell)
                            cell.value = translated

                # Restore column widths
                for col, width in col_widths.items():
                    ws.column_dimensions[col].width = width

            wb.save(dest_file)
            print(f"✅ Saved: {dest_file}")
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Done", f"🎉 Files split and formatting preserved!\nSaved to:\n{output_dir}",)
        temp_root.destroy()

    except Exception as e:
        temp_root = tk.Tk()
        temp_root.withdraw()
        messagebox.showinfo("Error", f"Error: {e}")
        temp_root.destroy()
        print("Exiting....")

if __name__ == "__main__":
    split_excel_preserving_format_and_formulas()
